# Tokenization & Embeddings Demo

## Learn: Byte-pair encoding, vocabulary building, token visualization

This notebook demonstrates:
1. **BPE Training**: Build subword vocabulary through iterative merging
2. **Encoding/Decoding**: Convert text to token IDs and back
3. **Token Visualization**: Understand token sequences
4. **Embedding Analysis**: One-hot vs learned embeddings, cosine similarity
5. **Compression Ablation**: How merges affect token compression

**Philosophy**: Code → Plot → Break → Learn. No theory without implementation.

In [ ]:
# Imports
from bpe import BPE
from embeddings import TokenEmbedder
import matplotlib.pyplot as plt
import numpy as np

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

: 

## 1. BPE Training & Vocabulary Building

BPE works by:
1. Start with character-level tokens
2. Repeatedly merge the most frequent pair of adjacent tokens
3. Repeat until desired vocabulary size

This learns a subword vocabulary that balances compression and expressiveness.

In [ ]:
# Sample corpus
corpus = [
    "hello", "world", "help", "held", "world",
    "data", "database", "day", "way", "play",
    "there", "where", "here", "their"
]

print(f"Corpus size: {len(corpus)} words")
print(f"Corpus: {corpus}")

: 

In [ ]:
# Train BPE with 20 merges
bpe = BPE(merges=20)
rules = bpe.train(corpus)

print(f"\nBPE Training Results:")
print(f"="*60)
print(f"Vocabulary size: {len(bpe.vocab)}")
print(f"Number of merge rules applied: {len(rules)}")
print(f"\nTop 10 tokens by frequency:")
top_tokens = sorted(bpe.vocab.items(), key=lambda x: x[1], reverse=True)[:10]
for token, freq in top_tokens:
    print(f"  {str(token):<20}: {freq:3d} occurrences")

In [ ]:
# Plot vocabulary statistics
bpe.plot_vocab_stats()

In [ ]:
# Plot merge history
bpe.plot_merge_history()

## 2. Encoding & Decoding

Once BPE is trained, we can encode text into token IDs and decode back.

In [ ]:
texts = [
    "hello world",
    "where is there",
    "playing in data"
]

for text in texts:
    token_ids = bpe.encode(text)
    decoded = bpe.decode(token_ids)
    
    print(f"\nText: '{text}'")
    print(f"Token IDs: {token_ids}")
    print(f"Decoded: '{decoded}'")
    print(f"Match: {decoded == text}")

## 3. Token Visualization

Visualize how text is split into tokens. This helps understand:
- Which subwords are created
- How token IDs map to actual tokens
- Compression ratio (chars → tokens)

In [ ]:
text = "the quick brown fox jumps over the lazy dog"
print(f"Visualizing tokens for: '{text}'")
bpe.token_ids_to_viz(text)

## 4. Embedding Space Analysis

Compare two ways to represent tokens:
- **One-hot**: Binary vector (1 at token position, 0 elsewhere). Orthogonal but doesn't capture similarity.
- **Learned**: Dense vector. Can capture semantic similarity through cosine distance.

In [ ]:
# Encode some text
text = "hello world help held where there"
token_ids = bpe.encode(text)

if len(token_ids) == 0:
    print("Could not encode text. Using first 10 tokens from vocab instead.")
    token_ids = list(range(min(10, len(bpe.vocab))))

print(f"Token IDs for analysis: {token_ids}")

# Create embedder
embedder = TokenEmbedder(len(bpe.vocab), embed_dim=64)
print(f"Created embedder with {len(bpe.vocab)} tokens, embedding dim=64")

In [ ]:
# Compare one-hot vs learned embeddings
print("\nComparing one-hot vs learned embeddings...")
embedder.compare_embeddings(token_ids)

In [ ]:
# Project learned embeddings to 2D using PCA
print("Projecting embeddings to 2D...")
embedder.visualize_embeddings_2d(token_ids)

In [ ]:
# Plot cosine similarity for learned embeddings
print("Plotting cosine similarity heatmap...")
embedder.plot_embedding_comparison(token_ids, 'learned')

## 5. Ablation: Merges vs Compression

How does the number of BPE merges affect compression ratio?

**Hypothesis**: More merges → better compression (smaller vocabulary means longer token sequences)

**Key Insight**: Diminishing returns - compression improves quickly at first, then plateaus

In [ ]:
test_text = "the quick brown fox jumps over the lazy dog and the brown fox runs"

compression_ratios = []
merge_counts = list(range(0, len(bpe.rules) + 1, 5))

print(f"Test text: '{test_text}'")
print(f"Text length: {len(test_text)} characters\n")

for num_merges in merge_counts:
    bpe_temp = BPE(merges=num_merges)
    bpe_temp.train(["hello", "world", "test"])
    bpe_temp.rules = bpe.rules[:num_merges]
    bpe_temp.token_to_id = bpe.token_to_id
    bpe_temp.id_to_token = bpe.id_to_token

    try:
        token_ids = bpe_temp.encode(test_text)
        ratio = 100 * len(token_ids) / len(test_text)
        compression_ratios.append(ratio)
        print(f"Merges: {num_merges:3d} | Tokens: {len(token_ids):4d} | Compression: {ratio:.1f}%")
    except:
        pass

In [ ]:
# Plot compression curve
if compression_ratios:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(merge_counts[:len(compression_ratios)], compression_ratios, 'o-', linewidth=2, markersize=8)
    ax.set_xlabel('Number of Merges', fontsize=12)
    ax.set_ylabel('Compression Ratio (%)', fontsize=12)
    ax.set_title('How BPE Merges Improve Compression', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Analysis
    print(f"\nCompression Analysis:")
    print(f"Initial (0 merges): {compression_ratios[0]:.1f}% (pure character tokens)")
    print(f"Final ({merge_counts[len(compression_ratios)-1]} merges): {compression_ratios[-1]:.1f}%")
    improvement = compression_ratios[0] - compression_ratios[-1]
    print(f"Total improvement: {improvement:.1f} percentage points")

## Key Insights

### What We Learned:

1. **BPE learns frequent patterns**: The algorithm discovers that common subwords (like "th", "er", "ing") should be merged into tokens

2. **Merges reduce redundancy**: Each merge reduces the number of character sequences needed

3. **Vocabulary vs Compression tradeoff**:
   - Larger vocabulary (many merges) → shorter token sequences but more tokens to store
   - Smaller vocabulary (few merges) → longer token sequences but fewer unique tokens

4. **Learned embeddings > one-hot**:
   - One-hot: All tokens are equally far apart (orthogonal)
   - Learned: Similar tokens can be close in embedding space

5. **Compression follows power law**: Improvement with more merges shows diminishing returns

### In Real LLMs:
- GPT-2: 50K tokens
- GPT-3: 50K tokens  
- LLaMA: 32K tokens
- Larger vocab → slower, larger models
- Smaller vocab → faster, more merges needed

### Try This Next:
- Train BPE with different corpus sizes
- Compare BPE vs WordPiece vs SentencePiece
- Analyze which subwords are most frequent
- See how tokenization affects downstream performance